In [116]:
# <editor-fold desc="making df">
import pandas as pd
import numpy as np
#import pandas as pd
import os
from sklearn.model_selection import train_test_split
#uploaded = files.upload()
#then change the filepath buffer once the file is in google.colab. also remove the file.upload() function

#name the column, adding informative names to features
#data has no header, and no footheader
#train= pd.merge(pd.read_csv('/content/test_features.csv'), pd.read_csv('/content/train_labels.csv'))
train_label_df = pd.merge(pd.read_csv('../m2w2 kaggle/train_features.csv'),  pd.read_csv('../m2w2 kaggle/train_labels.csv'))
#label =pd.read_csv('/content/train_labels.csv')
#train= pd.read_csv('/content/train_features.csv')
test_df = pd.read_csv('../m2w2 kaggle/test_features.csv')
submission_df = pd.read_csv('../m2w2 kaggle/sample_submission.csv')
# </editor-fold>


In [117]:
train_label_df.head(2)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [118]:
# <editor-fold desc="train, test, split">
from sklearn.model_selection import train_test_split
#Let Train, Validate, test then set features,target on .75 .25
train, validate = train_test_split(train_label_df, train_size = 0.75, test_size =0.25, random_state=42)
train.shape, validate.shape, test_df.shape
#I am using whole test_df to test. so let reword test_df to simply test
test = test_df
train.shape, validate.shape, test.shape
# </editor-fold>


((44550, 41), (14850, 41), (14358, 40))

In [93]:
# <editor-fold desc="wrangle function">
import numpy as np
def wrangle(X):
  #I will set boundries latitude (25,33), longitude (-14, 0)
  X=X.copy()
  #use number inbetween Long(25,44)   Lat (-14,0)     # can also use the replace method. many ways to do this.
  ###############################################################################
  #mask = ((X['longitude'] >= 22) & (X['longitude']<=44)) | ((X['latitude'] >= 1) & (X['latitude']<=-20))
  #X = X.loc[mask]
  ######################################################################
  X['latitude'] = X['latitude'].replace(-2e-08, 0)
    
    # When columns have zeros and shouldn't, they are like null values.
    # So we will replace the zeros with nulls, and impute missing values later.
  cols_with_zeros = ['longitude', 'latitude']
  for col in cols_with_zeros:
    X[col] = X[col].replace(0, np.nan)
  #rename 'status_group'
  X = X.rename(columns={'status_group':'status'})

  #Filter the dataframe with columns that has _group or _type in the syntax, then only list these columns.
  #Then Drop them because columns with _group or _type are duplicate of the original name that doesn't have those substring
  X = X.drop(X.filter(regex='_group').columns, axis=1)
  X = X.drop(X.filter(regex='_type').columns, axis=1)

  X = X.drop(columns=['basin', 'region', 'public_meeting', 'recorded_by', 'scheme_management', 'permit', 'management','quantity'])
  return X
# </editor-fold>

In [94]:
# <editor-fold desc="using wrangle function">
train = wrangle(train)
validate = wrangle(validate)
test = wrangle(test)
# </editor-fold>


In [95]:
# <editor-fold desc="target - features">
target = 'status'

train_features = train.drop(columns=[target,'id'])
#In the dataframe, grab all the numeric columns, then list it into columns. Finally turn it into array.
numeric_features = train_features.select_dtypes(include='number').columns.tolist()
#This (not a number) columns has how many unique categories?
cardinality = train_features.select_dtypes(exclude='number').nunique()
#if this columns has less than 50 keep that column, and turn it into array. (important step for panda ARRAY)
categorical_features = cardinality[cardinality <= 50].index.tolist()
#combine the list
features = numeric_features + categorical_features

# </editor-fold>

In [96]:
# <editor-fold desc="X-y train, X-y Validtae, test">
X_train = train[features]
y_train = train[target]

X_validate = train[features]
y_validate = train[target]

X_test = test[features]
# </editor-fold>


In [152]:
# <editor-fold desc="OneHot, SelectKbest,Ridge">
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import Ridge # add penalty equivalent to square of the magnitude of the coefficients
#So ridge regression puts constraint on the coefficients
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score


pipeline_Gridsearch_Randomized = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True),
    SimpleImputer(),
    StandardScaler(),
    SelectKBest(f_regression),
    Ridge()
)
#search will randomly select n_iter from this universe over cv=5 (times).
param_distributions = {
    'simpleimputer__strategy' : ['mean','median'],
    'selectkbest__k' : range(1, len(X_train.columns)+1),
    'ridge__alpha': [0.1,1,10],
}
search = RandomizedSearchCV(
    pipeline_Gridsearch_Randomized,
    param_distributions=param_distributions,
    n_iter=30,
    cv=5,
    verbose=2,
    return_train_score=False,
    n_jobs=-1
)
# </editor-fold>
#search.fit(X_train, y_train);


In [154]:
# <editor-fold desc="Target encoder, SimpleImputer">
from scipy.stats import randint, uniform
from sklearn.ensemble import RandomForestRegressor
pipeline_TargetEncode_RandomForest = make_pipeline(
    ce.TargetEncoder(),
    SimpleImputer(),
    RandomForestRegressor(random_state=42)
)

param_distributions_Target_RandomForest = {
    'targetencoder__min_samples_leaf' : randint(1,1000),
    'targetencoder__smoothing' : uniform(1,1000),
    'simpleimputer__strategy': ['mean', 'median'],
    'randomforestregressor__n_estimators' : randint(50,500),
    'randomforestregressor__max_depth':[5,10,15,20,None],
    'randomforestregressor__max_features': uniform(0,1),
}
#multi_class='auto',solver='lbfgs',cv=5,n_jobs=-1
search_Target_random = RandomizedSearchCV(
    pipeline_TargetEncode_RandomForest,
    param_distributions=param_distributions_Target_RandomForest,
    n_iter=10,
    cv=3,
    scoring='neg_mean_absolute_error’, verbose=10,
    return_train_score=True,
    n_jobs=-1
)
# </editor-fold>

SyntaxError: EOL while scanning string literal (<ipython-input-154-f77cc9a3f529>, line 24)

In [138]:
# <editor-fold desc="OneHot, Select, Ridge">
import category_encoders as ce
import numpy as np
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score


pipeline_OneHot_KBest_Ridge = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True),
    SimpleImputer(strategy='mean'),
    StandardScaler(),
    SelectKBest(f_regression, k=20),
    Ridge(alpha=1.0)
)
#k=3
#scores = accuracy_score( pipeline_OneHot_KBest_Ridge, X_train, y_train)
# </editor-fold>

In [155]:
search.score(X_train, y_train)


NotFittedError: This RandomizedSearchCV instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [148]:
pipeline_TargetEncode_RandomForest.fit(X_train, y_train)



TypeError: Could not convert non functionalfunctionalfunctionalfunctionalnon functionalfunctionalfunctionalfunctionalnon functionalnon functionalfunctionalfunctionalnon functionalfunctionalfunctionalfunctionalfunctionalnon functionalfunctionalnon functionalfunctionalfunctionalnon functionalnon functionalnon functionalnon functionalfunctionalnon functionalnon functionalfunctionalfunctionalfunctionalfunctionalnon functionalfunctionalfunctionalfunctionalnon functionalfunctionalfunctionalfunctionalfunctionalfunctionalnon functionalfunctionalfunctionalnon functionalfunctionalfunctionalfunctionalnon functionalnon functionalfunctional needs repairnon functionalfunctionalnon functionalfunctional needs repairnon functionalnon functionalfunctionalnon functionalnon functionalfunctionalfunctionalnon functionalfunctionalfunctionalnon functionalnon functionalfunctionalfunctionalnon functionalnon functionalfunctionalnon functionalnon functionalfunctionalnon functionalnon functionalfunctionalnon funct

In [150]:
search.fit(X_train, y_train);
print(search.best_params_)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    5.1s


TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [151]:
print(search.best_params_)



AttributeError: 'RandomizedSearchCV' object has no attribute 'best_params_'